In [13]:
# for validation
import numpy as np
import pandas as pd
import datetime
from netCDF4 import Dataset,num2date,date2num
import gpflow as gf
from eofs.standard import Eof
import time
from datetime import datetime, timedelta
import numpy.ma as ma
import os
from typing import Sequence
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [14]:
# Load historical simulation data
historical_simulation_data = Dataset('D:\CWatM-new\Brahmaputra\output227\discharge_daily_gfdl.nc','r')

# Load historical observation data
historical_observation_data = Dataset('E:\data\era5_runoff\disdata\era5_data1979-2014_5min.nc','r')

gfdl_hist_train_origin = historical_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

obsdata_origin = historical_observation_data.variables['dis24'][:]
historical_observation_data.close()
historical_simulation_data.close()

mask_t = gfdl_hist_train_origin == gfdl_hist_train_origin[1][0][0]
gfdl_hist_train = ma.array(gfdl_hist_train_origin,mask = mask_t)
obsdata = ma.array(obsdata_origin,mask = mask_t)
#数据切片调试-可以不做
#gfdl_hist_train_old = ma.array(gfdl_hist_train_origin,mask = mask_t)
#obsdata_old = ma.array(obsdata_origin,mask = mask_t)
#gfdl_hist_train = gfdl_hist_train_old[13000:][:][:]
#obsdata = obsdata_old[13000:][:][:]

In [15]:
#提取两个数据的主成分，减少分析两个数据差异的复杂度
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
#coslat = np.cos(np.deg2rad(lats)).clip(0., 1.)
#wgts = np.sqrt(coslat)[..., np.newaxis]


# Perform EOF analysis on data1 and data2
#obsdata_solver = Eof(obsdata, weights=None, center=False)
obsdata_solver = Eof(obsdata, weights=None,center=False)

#explain for how much variance
obsdata_mode = obsdata_solver.varianceFraction(neigs=4)
print(obsdata_mode)

# Retrieve the leading EOFs for both datasets
obsdata_eofs = obsdata_solver.eofs(neofs=4)
print(obsdata_eofs.shape)

#variance correlation
obsdata_eofc = obsdata_solver.eofsAsCorrelation(neofs=4)


#create pseudo-ECs for GPR

#源代码中obsdata 不存在 obsdata -> obsdata_old
gfdl_hist_train_ECs = obsdata_solver.projectField(gfdl_hist_train,neofs = 4)
obsdata_ECs = obsdata_solver.projectField(obsdata,neofs = 4)
print(gfdl_hist_train_ECs.shape)

[0.9868846  0.0065986  0.00354106 0.00170928]
(4, 89, 189)
(13149, 4)


In [16]:
#将两列ECs各自做为训练集
gfdl_hist_train_ECs_l1 = gfdl_hist_train_ECs[:,0].reshape(13149,1)
gfdl_hist_train_ECs_x1 = tf.convert_to_tensor(gfdl_hist_train_ECs_l1)
gfdl_hist_train_ECs_x1=tf.cast(gfdl_hist_train_ECs_x1,tf.float64)
gfdl_hist_train_ECs_l2 = gfdl_hist_train_ECs[:,1].reshape(13149,1)
gfdl_hist_train_ECs_x2 = tf.convert_to_tensor(gfdl_hist_train_ECs_l2)
gfdl_hist_train_ECs_x2=tf.cast(gfdl_hist_train_ECs_x2,tf.float64)
gfdl_hist_train_ECs_l3 = gfdl_hist_train_ECs[:,2].reshape(13149,1)
gfdl_hist_train_ECs_x3 = tf.convert_to_tensor(gfdl_hist_train_ECs_l3)
gfdl_hist_train_ECs_x3=tf.cast(gfdl_hist_train_ECs_x3,tf.float64)
gfdl_hist_train_ECs_l4 = gfdl_hist_train_ECs[:,3].reshape(13149,1)
gfdl_hist_train_ECs_x4 = tf.convert_to_tensor(gfdl_hist_train_ECs_l4)
gfdl_hist_train_ECs_x4=tf.cast(gfdl_hist_train_ECs_x4,tf.float64)
obsdata_ECs_l1 = obsdata_ECs[:,0].reshape(13149,1)
obsdata_ECs_y1 = tf.convert_to_tensor(obsdata_ECs_l1)
obsdata_ECs_y1=tf.cast(obsdata_ECs_y1,tf.float64)
obsdata_ECs_l2 = obsdata_ECs[:,1].reshape(13149,1)
obsdata_ECs_y2 = tf.convert_to_tensor(obsdata_ECs_l2)
obsdata_ECs_y2=tf.cast(obsdata_ECs_y2,tf.float64)
obsdata_ECs_l3 = obsdata_ECs[:,2].reshape(13149,1)
obsdata_ECs_y3 = tf.convert_to_tensor(obsdata_ECs_l3)
obsdata_ECs_y3=tf.cast(obsdata_ECs_y3,tf.float64)
obsdata_ECs_l4 = obsdata_ECs[:,3].reshape(13149,1)
obsdata_ECs_y4 = tf.convert_to_tensor(obsdata_ECs_l4)
obsdata_ECs_y4=tf.cast(obsdata_ECs_y4,tf.float64)

In [17]:
#EOF1
#standardize x,y
n_input_ECs = 4
scaler_x = StandardScaler()
scaler_y = StandardScaler()
gfdl_train = np.c_[gfdl_hist_train_ECs_x1,gfdl_hist_train_ECs_x2,gfdl_hist_train_ECs_x3,gfdl_hist_train_ECs_x4]
gfdl_train_sc = scaler_x.fit_transform(gfdl_train)
#inducing（修改）
n_inducing_points=700
model_gfdl_eof = list()
obs_train_all = np.c_[obsdata_ECs_y1,obsdata_ECs_y2,obsdata_ECs_y3,obsdata_ECs_y4]
for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(gfdl_train_sc[:,0].min(), gfdl_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(gfdl_train_sc[:, j].min(), gfdl_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(gfdl_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (gfdl_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_gfdl_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_gfdl_eof[i].training_loss, model_gfdl_eof[i].trainable_variables)
    gf.utilities.print_summary(model_gfdl_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_gfdl_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_gfdl_eof1_timeperiod:',opt_gfdl_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_gfdl_eof[i].predict_y(gfdl_train_sc, full_cov=False)
    gfdl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(gfdl_pred_mean) / np.mean(obs_train)
    y = (np.std(gfdl_pred_mean) / np.mean(gfdl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.93436
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.82767
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.24122
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-7.21994, -4.33386, -7.17832..."


The  1  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 27660.78099999996
0.998309381360726 0.8653666363935465


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.16208
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.73013
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.56107
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-7.15535, -5.05674, -6.29757..."


The  2  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 18336.563000000024
1.007360947031738 0.6317288755692053


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.62939
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.6011
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.77659
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-6.93874, -4.81028, -6.84591..."


The  3  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 10899.108999999997
0.985243524728622 0.42770509054320216


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.5814
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.63184
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.75717
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-8.66647, -3.61433, -7.36221..."


The  4  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 12340.829000000027
1.0265947486627987 0.43771812164103086


In [18]:
gfdl_pred_ECs = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_lb = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

for i in range(n_input_ECs):
    y_pred_mean, y_pred_var= model_gfdl_eof[i].predict_y(gfdl_train_sc, full_cov=False)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    print(y_pred_mean,obs_train_sc)
    gfdl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    print(gfdl_pred_mean,obs_train)
    B = np.mean(gfdl_pred_mean) / np.mean(obs_train)
    y = (np.std(gfdl_pred_mean) / np.mean(gfdl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)
    gfdl_pred_ECs[:,i] =np.squeeze(gfdl_pred_mean)
    print(gfdl_pred_ECs[:,i])
    y__pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    gfdl_pred_ECs_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    gfdl_pred_ECs_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
gfdl_hist_train_ECs_SGPRcor=tf.concat([gfdl_pred_ECs[:,0].reshape(13149, 1),gfdl_pred_ECs[:,1].reshape(13149, 1),gfdl_pred_ECs[:,2].reshape(13149, 1),gfdl_pred_ECs[:,3].reshape(13149, 1)],axis=1)
gfdl_hist_train_ECs_lb_SGPRcor=tf.concat([gfdl_pred_ECs_lb[:,0].reshape(13149, 1),gfdl_pred_ECs_lb[:,1].reshape(13149, 1),gfdl_pred_ECs_lb[:,2].reshape(13149, 1),gfdl_pred_ECs_lb[:,3].reshape(13149, 1)],axis=1)
gfdl_hist_train_ECs_ub_SGPRcor=tf.concat([gfdl_pred_ECs_ub[:,0].reshape(13149, 1),gfdl_pred_ECs_ub[:,1].reshape(13149, 1),gfdl_pred_ECs_ub[:,2].reshape(13149, 1),gfdl_pred_ECs_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(gfdl_hist_train_ECs_SGPRcor,columns = ['gfdl_EC1','gfdl_EC2','gfdl_EC3','gfdl_EC4'])
df_lb_merge = pd.DataFrame(gfdl_hist_train_ECs_lb_SGPRcor,columns = ['gfdl_EC1_lb','gfdl_EC2_lb','gfdl_EC3_lb','gfdl_EC4_lb'])
df_ub_merge = pd.DataFrame(gfdl_hist_train_ECs_ub_SGPRcor,columns = ['gfdl_EC1_ub','gfdl_EC2_ub','gfdl_EC3_ub','gfdl_EC4_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

tf.Tensor(
[[0.87331546]
 [0.87726414]
 [0.8770928 ]
 ...
 [0.88052753]
 [0.88104039]
 [0.88143235]], shape=(13149, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[[-35288.93138563]
 [-34739.26888568]
 [-34763.11949319]
 ...
 [-34285.0002461 ]
 [-34213.60946896]
 [-34159.04810806]] [[-26780.41015625]
 [-26305.3046875 ]
 [-25864.76757812]
 ...
 [-31726.390625  ]
 [-31462.87109375]
 [-31136.0703125 ]]
0.998309381360726 0.8653666363935465
[-35288.93138563 -34739.26888568 -34763.11949319 ... -34285.0002461
 -34213.60946896 -34159.04810806]
tf.Tensor(
[[ 0.01076072]
 [ 0.00352148]
 [ 0.00261636]
 ...
 [-0.00904002]
 [-0.00616283]
 [-0.00335035]], shape=(13149, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[[-188.82933555]
 [-312.9415743 ]
 [-328.45925931]
 ...
 [-528.3005516 ]
 [-478.97287184]
 [-430.75481779]] [[-2039.22070312]
 [-1982.69091797]
 [-1931.4150390

In [19]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
gfdl_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_gfdl_eof_2GP = list()
gfdl_pred_ECs_range = np.zeros((13149,n_input_ECs))
for i in range(n_input_ECs):
    gfdl_pred_ECs_range[:,i] = np.subtract(gfdl_pred_ECs_ub[:,i],gfdl_pred_ECs_lb[:,i])
    gfdl_train_2GP = np.c_[gfdl_pred_ECs[:,i],gfdl_pred_ECs_lb[:,i],gfdl_pred_ECs_ub[:,i],gfdl_pred_ECs_range[:,i] ]
    gfdl_train_2GP_sc = scaler_z.fit_transform(gfdl_train_2GP)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(gfdl_train_2GP_sc[:,0].min(), gfdl_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(gfdl_train_2GP_sc[:, j].min(), gfdl_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(gfdl_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (gfdl_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_gfdl_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_gfdl_eof_2GP[i].training_loss, model_gfdl_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_gfdl_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_gfdl_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_gfdl_eof1_2GP_timeperiod:',opt_gfdl_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_gfdl_eof_2GP[i].predict_y(gfdl_train_2GP_sc, full_cov=False)
    gfdl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(gfdl_pred_mean) / np.mean(obs_train)
    y = (np.std(gfdl_pred_mean) / np.mean(gfdl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    gfdl_pred_ECs_2GP[:,i] =np.squeeze(gfdl_pred_mean)
    print(gfdl_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    gfdl_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    gfdl_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,172.04203
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,39.64593
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.22768
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.27071, -2.46058, -3.04691..."


The  1  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 12340.829000000027
[-36423.94743507 -35919.67355817 -35943.77676201 ... -35478.40766452
 -35413.21957169 -35363.53152788]
0.999998267413418 0.8787575420541192


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,6.92879
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,13.78064
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.52331
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.56606, -4.35139, -4.58452..."


The  2  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 12340.829000000027
[-208.18256854 -340.23371314 -356.9085405  ... -566.04678982 -513.77387892
 -462.6832967 ]
0.9998347919427906 0.6903304281250986


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,14.10745
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,12.13212
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.71575
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.82574, -4.70984, -5.04268..."


The  3  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 12340.829000000027
[2384.21039485 2421.3337729  2425.59448927 ... 2896.64143875 2885.78952986
 2875.020921  ]
1.0000081966398797 0.5327270970943646


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,4.45698
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,4.67619
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.70175
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.32106, -7.31641, -4.22884..."


The  4  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 12340.829000000027
[-173.48829673 -318.01283417 -345.47255413 ... -205.23462891 -197.60892631
 -188.05002406]
0.9999633471463144 0.5456627345912679


In [20]:
#gfdl_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
#gfdl_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

#for i in range(n_input_ECs):
#    y_pred_mean, y_pred_var= model_gfdl_eof_2GP[i].predict_y(gfdl_train_2GP_sc, full_cov=False)
#    obs_train = obs_train_all[:,i].reshape(13149,1)
#    obs_train_sc = scaler_y.fit_transform(obs_train)
#    print(y_pred_mean,obs_train_sc)
#    gfdl_pred_mean_2GP = scaler_y.inverse_transform(y_pred_mean)
#    print(gfdl_pred_mean_2GP,obs_train)
#    B = np.mean(gfdl_pred_mean_2GP) / np.mean(obs_train)
#    y = (np.std(gfdl_pred_mean_2GP) / np.mean(gfdl_pred_mean_2GP)) / (np.std(obs_train) / np.mean(obs_train))
#    print(B,y)
#    gfdl_pred_ECs_2GP[:,i] =np.squeeze(gfdl_pred_mean_2GP)
#    print(gfdl_pred_ECs_2GP[:,i])
#    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
#    y_pred_lb = y_pred_mean - y__pred_conf
#    y_pred_ub = y_pred_mean + y__pred_conf
#    gfdl_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
#    gfdl_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
gfdl_hist_train_ECs_2GP_SGPRcor=tf.concat([gfdl_pred_ECs_2GP[:,0].reshape(13149, 1),gfdl_pred_ECs_2GP[:,1].reshape(13149, 1),gfdl_pred_ECs_2GP[:,2].reshape(13149, 1),gfdl_pred_ECs_2GP[:,3].reshape(13149, 1)],axis=1)
gfdl_hist_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_lb[:,0].reshape(13149, 1),gfdl_pred_ECs_2GP_lb[:,1].reshape(13149, 1),gfdl_pred_ECs_2GP_lb[:,2].reshape(13149, 1),gfdl_pred_ECs_2GP_lb[:,3].reshape(13149, 1)],axis=1)
gfdl_hist_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ub[:,0].reshape(13149, 1),gfdl_pred_ECs_2GP_ub[:,1].reshape(13149, 1),gfdl_pred_ECs_2GP_ub[:,2].reshape(13149, 1),gfdl_pred_ECs_2GP_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(gfdl_hist_train_ECs_2GP_SGPRcor,columns = ['gfdl_EC1_2GP','gfdl_EC2_2GP','gfdl_EC3_2GP','gfdl_EC4_2GP'])
df_lb_merge = pd.DataFrame(gfdl_hist_train_ECs_2GP_lb_SGPRcor,columns = ['gfdl_EC1_2GP_lb','gfdl_EC2_2GP_lb','gfdl_EC3_2GP_lb','gfdl_EC4_2GP_lb'])
df_ub_merge = pd.DataFrame(gfdl_hist_train_ECs_2GP_ub_SGPRcor,columns = ['gfdl_EC1_2GP_ub','gfdl_EC2_2GP_ub','gfdl_EC3_2GP_ub','gfdl_EC4_2GP_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_2GP.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_2GP.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_2GP.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

In [21]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((13149,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [22]:
recondata_SGPR_gfdl_mix4_2GP = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_hist_train_ECs_2GP_SGPRcor)

In [23]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(1979, 1, 1)
    end_date = datetime(2014, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 1979-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [24]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc = out_dir+'recondata_SGPR_gfdl_mix4_2GP.nc' 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP,out_file_nc) 

# 做未来预测

In [25]:
#读入数据
ssp126_gfdl_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_gfdl_ssp126.nc','r')
ssp370_gfdl_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_gfdl_ssp370.nc','r')
ssp585_gfdl_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_gfdl_ssp585.nc','r')

gfdl_ssp126_train_origin = ssp126_gfdl_simulation_data.variables['discharge'][:]
gfdl_ssp370_train_origin = ssp370_gfdl_simulation_data.variables['discharge'][:]
gfdl_ssp585_train_origin = ssp585_gfdl_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

mask_t = gfdl_ssp126_train_origin == gfdl_ssp126_train_origin[1][0][0]

gfdl_ssp126_train = ma.array(gfdl_ssp126_train_origin,mask = mask_t)
gfdl_ssp370_train = ma.array(gfdl_ssp370_train_origin,mask = mask_t)
gfdl_ssp585_train = ma.array(gfdl_ssp585_train_origin,mask = mask_t)

gfdl_ssp126_train_ECs = obsdata_solver.projectField(gfdl_ssp126_train,neofs = 4)
gfdl_ssp370_train_ECs = obsdata_solver.projectField(gfdl_ssp370_train,neofs = 4)
gfdl_ssp585_train_ECs = obsdata_solver.projectField(gfdl_ssp585_train,neofs = 4)

ssp126_gfdl_simulation_data.close()
ssp370_gfdl_simulation_data.close()
ssp585_gfdl_simulation_data.close()

In [26]:
#将两列ECs各自做为训练集
gfdl_ssp126_train_ECs_l1 = gfdl_ssp126_train_ECs[:,0].reshape(28854,1)
gfdl_ssp126_train_ECs_x1 = tf.convert_to_tensor(gfdl_ssp126_train_ECs_l1)
gfdl_ssp126_train_ECs_x1=tf.cast(gfdl_ssp126_train_ECs_x1,tf.float64)
gfdl_ssp126_train_ECs_l2 = gfdl_ssp126_train_ECs[:,1].reshape(28854,1)
gfdl_ssp126_train_ECs_x2 = tf.convert_to_tensor(gfdl_ssp126_train_ECs_l2)
gfdl_ssp126_train_ECs_x2=tf.cast(gfdl_ssp126_train_ECs_x2,tf.float64)
gfdl_ssp126_train_ECs_l3 = gfdl_ssp126_train_ECs[:,2].reshape(28854,1)
gfdl_ssp126_train_ECs_x3 = tf.convert_to_tensor(gfdl_ssp126_train_ECs_l3)
gfdl_ssp126_train_ECs_x3=tf.cast(gfdl_ssp126_train_ECs_x3,tf.float64)
gfdl_ssp126_train_ECs_l4 = gfdl_ssp126_train_ECs[:,3].reshape(28854,1)
gfdl_ssp126_train_ECs_x4 = tf.convert_to_tensor(gfdl_ssp126_train_ECs_l4)
gfdl_ssp126_train_ECs_x4=tf.cast(gfdl_ssp126_train_ECs_x4,tf.float64)
gfdl_ssp370_train_ECs_l1 = gfdl_ssp370_train_ECs[:,0].reshape(28854,1)
gfdl_ssp370_train_ECs_x1 = tf.convert_to_tensor(gfdl_ssp370_train_ECs_l1)
gfdl_ssp370_train_ECs_x1=tf.cast(gfdl_ssp370_train_ECs_x1,tf.float64)
gfdl_ssp370_train_ECs_l2 = gfdl_ssp370_train_ECs[:,1].reshape(28854,1)
gfdl_ssp370_train_ECs_x2 = tf.convert_to_tensor(gfdl_ssp370_train_ECs_l2)
gfdl_ssp370_train_ECs_x2=tf.cast(gfdl_ssp370_train_ECs_x2,tf.float64)
gfdl_ssp370_train_ECs_l3 = gfdl_ssp370_train_ECs[:,2].reshape(28854,1)
gfdl_ssp370_train_ECs_x3 = tf.convert_to_tensor(gfdl_ssp370_train_ECs_l3)
gfdl_ssp370_train_ECs_x3=tf.cast(gfdl_ssp370_train_ECs_x3,tf.float64)
gfdl_ssp370_train_ECs_l4 = gfdl_ssp370_train_ECs[:,3].reshape(28854,1)
gfdl_ssp370_train_ECs_x4 = tf.convert_to_tensor(gfdl_ssp370_train_ECs_l4)
gfdl_ssp370_train_ECs_x4=tf.cast(gfdl_ssp370_train_ECs_x4,tf.float64)
gfdl_ssp585_train_ECs_l1 = gfdl_ssp585_train_ECs[:,0].reshape(28854,1)
gfdl_ssp585_train_ECs_x1 = tf.convert_to_tensor(gfdl_ssp585_train_ECs_l1)
gfdl_ssp585_train_ECs_x1=tf.cast(gfdl_ssp585_train_ECs_x1,tf.float64)
gfdl_ssp585_train_ECs_l2 = gfdl_ssp585_train_ECs[:,1].reshape(28854,1)
gfdl_ssp585_train_ECs_x2 = tf.convert_to_tensor(gfdl_ssp585_train_ECs_l2)
gfdl_ssp585_train_ECs_x2=tf.cast(gfdl_ssp585_train_ECs_x2,tf.float64)
gfdl_ssp585_train_ECs_l3 = gfdl_ssp585_train_ECs[:,2].reshape(28854,1)
gfdl_ssp585_train_ECs_x3 = tf.convert_to_tensor(gfdl_ssp585_train_ECs_l3)
gfdl_ssp585_train_ECs_x3=tf.cast(gfdl_ssp585_train_ECs_x3,tf.float64)
gfdl_ssp585_train_ECs_l4 = gfdl_ssp585_train_ECs[:,3].reshape(28854,1)
gfdl_ssp585_train_ECs_x4 = tf.convert_to_tensor(gfdl_ssp585_train_ECs_l4)
gfdl_ssp585_train_ECs_x4=tf.cast(gfdl_ssp585_train_ECs_x4,tf.float64)

In [27]:
gfdl_pred_ECs_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_lb_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_ub_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_lb_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_ub_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_ssp585 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_lb_ssp585 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_ub_ssp585 = np.zeros((28854,n_input_ECs))
scaler_x_ssp126 = StandardScaler()
gfdl_train_ssp126 = np.c_[gfdl_ssp126_train_ECs_x1,gfdl_ssp126_train_ECs_x2,gfdl_ssp126_train_ECs_x3,gfdl_ssp126_train_ECs_x4]
gfdl_train_ssp126_sc = scaler_x_ssp126.fit_transform(gfdl_train_ssp126)
scaler_x_ssp370 = StandardScaler()
gfdl_train_ssp370 = np.c_[gfdl_ssp370_train_ECs_x1,gfdl_ssp370_train_ECs_x2,gfdl_ssp370_train_ECs_x3,gfdl_ssp370_train_ECs_x4]
gfdl_train_ssp370_sc = scaler_x_ssp370.fit_transform(gfdl_train_ssp370)
scaler_x_ssp585 = StandardScaler()
gfdl_train_ssp585 = np.c_[gfdl_ssp585_train_ECs_x1,gfdl_ssp585_train_ECs_x2,gfdl_ssp585_train_ECs_x3,gfdl_ssp585_train_ECs_x4]
gfdl_train_ssp585_sc = scaler_x_ssp585.fit_transform(gfdl_train_ssp585)
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

In [28]:
#EOF1

for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(gfdl_train_sc[:,0].min(), gfdl_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(gfdl_train_sc[:, j].min(), gfdl_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(gfdl_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (gfdl_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_gfdl_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_gfdl_eof[i].training_loss, model_gfdl_eof[i].trainable_variables)
    gf.utilities.print_summary(model_gfdl_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_gfdl_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_gfdl_eof1_timeperiod:',opt_gfdl_eof1_timeperiod)
    y_pred_mean_ssp126, y_pred_var_ssp126= model_gfdl_eof[i].predict_y(gfdl_train_ssp126_sc, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    gfdl_pred_mean_ssp126 = scaler_y.inverse_transform(y_pred_mean_ssp126)
    gfdl_pred_ECs_ssp126[:,i] =np.squeeze(gfdl_pred_mean_ssp126)
    print(gfdl_pred_ECs_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    gfdl_pred_ECs_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    gfdl_pred_ECs_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_ssp370, y_pred_var_ssp370= model_gfdl_eof[i].predict_y(gfdl_train_ssp370_sc, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    gfdl_pred_mean_ssp370 = scaler_y.inverse_transform(y_pred_mean_ssp370)
    gfdl_pred_ECs_ssp370[:,i] =np.squeeze(gfdl_pred_mean_ssp370)
    print(gfdl_pred_ECs_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    gfdl_pred_ECs_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    gfdl_pred_ECs_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_ssp585, y_pred_var_ssp585= model_gfdl_eof[i].predict_y(gfdl_train_ssp585_sc, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    gfdl_pred_mean_ssp585 = scaler_y.inverse_transform(y_pred_mean_ssp585)
    gfdl_pred_ECs_ssp585[:,i] =np.squeeze(gfdl_pred_mean_ssp585)
    print(gfdl_pred_ECs_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    gfdl_pred_ECs_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    gfdl_pred_ECs_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))
    y_pred_mean, y_pred_var = model_gfdl_eof[i].predict_y(gfdl_train_sc, full_cov=False)
    gfdl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(gfdl_pred_mean) / np.mean(obs_train)
    y = (np.std(gfdl_pred_mean) / np.mean(gfdl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.93439
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.82587
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.24105
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-7.21974, -4.33363, -7.17724..."


The  1  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 5361.7970000000205
tf.Tensor(
[[ 0.6178286 ]
 [ 0.39492175]
 [-0.07511221]
 ...
 [ 0.77027258]
 [ 0.77023625]
 [ 0.77033459]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[ -70853.10909952 -101882.09796853 -167311.5733599  ...  -49632.66486114
  -49637.72186343  -49624.03261617]
tf.Tensor(
[[ 0.54020024]
 [ 0.2506153 ]
 [-0.13276278]
 ...
 [ 0.71084329]
 [ 0.71331597]
 [ 0.71576031]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[ -81659.10120835 -121969.78515368 -175336.62526224 ...  -57905.31605365
  -57561.1157227   -57220.86005215]
tf.Tensor(
[[ 0.49714996]
 [ 0.26525922]
 [-0.1110656 ]
 ...
 [ 0.71057343]
 [ 0.7090416 ]
 [ 0.70960243]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[ -87651.76

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.16208
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.73013
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.56106
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-7.15535, -5.05674, -6.29757..."


The  2  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 18.875
tf.Tensor(
[[0.17736323]
 [0.42719593]
 [0.90959215]
 ...
 [0.28288778]
 [0.28648861]
 [0.28953977]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 2667.46509553  6950.68859167 15221.06619439 ...  4476.61665806
  4538.35068257  4590.6608185 ]
tf.Tensor(
[[0.3192296 ]
 [0.64891987]
 [1.00152113]
 ...
 [0.35928764]
 [0.36385522]
 [0.36828352]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 5099.67424366 10752.00495513 16797.13025656 ...  5786.44388947
  5864.75214268  5940.67261243]
tf.Tensor(
[[0.34052321]
 [0.60681998]
 [0.97621153]
 ...
 [0.29770499]
 [0.31061083]
 [0.32296627]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 5464.73958922 10030.2290555  16363.21326329 .

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.62939
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.6011
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.77659
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-6.93874, -4.81028, -6.84591..."


The  3  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 18.54699999996228
tf.Tensor(
[[-0.04932786]
 [-0.16087757]
 [-0.49653032]
 ...
 [ 0.14905888]
 [ 0.14994545]
 [ 0.15074681]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[  586.95728995  -807.90176711 -5005.02832252 ...  3067.65866413
  3078.7446143   3088.76515013]
tf.Tensor(
[[-0.0969394 ]
 [-0.22812809]
 [-0.51429297]
 ...
 [ 0.12866771]
 [ 0.13487639]
 [ 0.14099499]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[   -8.39514536 -1648.82723066 -5227.13902297 ...  2812.67987762
  2890.31550566  2966.824842  ]
tf.Tensor(
[[-0.14457556]
 [-0.25868827]
 [-0.51401258]
 ...
 [ 0.10838089]
 [ 0.11071252]
 [ 0.1149439 ]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -604.05536377 -2030.96304659 -5223

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.58139
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.63186
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.75721
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-8.66647, -3.61433, -7.36221..."


The  4  ECs training for obsdata
opt_gfdl_eof1_timeperiod: 22.40600000001723
tf.Tensor(
[[0.12659169]
 [0.44806331]
 [0.91077006]
 ...
 [0.07833322]
 [0.08099441]
 [0.08326765]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[2959.98167882 5699.78058497 9643.28083913 ... 2548.69031581 2571.37073444
 2590.74485754]
tf.Tensor(
[[0.30505153]
 [0.70401854]
 [0.9268653 ]
 ...
 [0.22959557]
 [0.23141436]
 [0.23332188]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[4480.93729582 7881.20437827 9780.45541986 ... 3837.85053006 3853.35145351
 3869.60860274]
tf.Tensor(
[[0.36931137]
 [0.69106013]
 [0.94284326]
 ...
 [0.18429146]
 [0.19881703]
 [0.21344734]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[5028.60318557 7770.76403839 9916.63038432 ... 3

In [29]:
#合并校正后的两个主成分-
gfdl_ssp126_train_ECs_SGPRcor=tf.concat([gfdl_pred_ECs_ssp126[:,0].reshape(28854, 1),gfdl_pred_ECs_ssp126[:,1].reshape(28854, 1),gfdl_pred_ECs_ssp126[:,2].reshape(28854, 1),gfdl_pred_ECs_ssp126[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp126_train_ECs_lb_SGPRcor=tf.concat([gfdl_pred_ECs_lb_ssp126[:,0].reshape(28854, 1),gfdl_pred_ECs_lb_ssp126[:,1].reshape(28854, 1),gfdl_pred_ECs_lb_ssp126[:,2].reshape(28854, 1),gfdl_pred_ECs_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp126_train_ECs_ub_SGPRcor=tf.concat([gfdl_pred_ECs_ub_ssp126[:,0].reshape(28854, 1),gfdl_pred_ECs_ub_ssp126[:,1].reshape(28854, 1),gfdl_pred_ECs_ub_ssp126[:,2].reshape(28854, 1),gfdl_pred_ECs_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp370_train_ECs_SGPRcor=tf.concat([gfdl_pred_ECs_ssp370[:,0].reshape(28854, 1),gfdl_pred_ECs_ssp370[:,1].reshape(28854, 1),gfdl_pred_ECs_ssp370[:,2].reshape(28854, 1),gfdl_pred_ECs_ssp370[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp370_train_ECs_lb_SGPRcor=tf.concat([gfdl_pred_ECs_lb_ssp370[:,0].reshape(28854, 1),gfdl_pred_ECs_lb_ssp370[:,1].reshape(28854, 1),gfdl_pred_ECs_lb_ssp370[:,2].reshape(28854, 1),gfdl_pred_ECs_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp370_train_ECs_ub_SGPRcor=tf.concat([gfdl_pred_ECs_ub_ssp370[:,0].reshape(28854, 1),gfdl_pred_ECs_ub_ssp370[:,1].reshape(28854, 1),gfdl_pred_ECs_ub_ssp370[:,2].reshape(28854, 1),gfdl_pred_ECs_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp585_train_ECs_SGPRcor=tf.concat([gfdl_pred_ECs_ssp585[:,0].reshape(28854, 1),gfdl_pred_ECs_ssp585[:,1].reshape(28854, 1),gfdl_pred_ECs_ssp585[:,2].reshape(28854, 1),gfdl_pred_ECs_ssp585[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp585_train_ECs_lb_SGPRcor=tf.concat([gfdl_pred_ECs_lb_ssp585[:,0].reshape(28854, 1),gfdl_pred_ECs_lb_ssp585[:,1].reshape(28854, 1),gfdl_pred_ECs_lb_ssp585[:,2].reshape(28854, 1),gfdl_pred_ECs_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp585_train_ECs_ub_SGPRcor=tf.concat([gfdl_pred_ECs_ub_ssp585[:,0].reshape(28854, 1),gfdl_pred_ECs_ub_ssp585[:,1].reshape(28854, 1),gfdl_pred_ECs_ub_ssp585[:,2].reshape(28854, 1),gfdl_pred_ECs_ub_ssp585[:,3].reshape(28854, 1)],axis=1)



In [35]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
gfdl_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_gfdl_eof_2GP = list()
gfdl_pred_ECs_range = np.zeros((13149,n_input_ECs))
gfdl_pred_ECs_2GP_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_lb_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_ub_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_range_ssp126 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_lb_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_ub_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_range_ssp370 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_ssp585 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_lb_ssp585 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_2GP_ub_ssp585 = np.zeros((28854,n_input_ECs))
gfdl_pred_ECs_range_ssp585 = np.zeros((28854,n_input_ECs))
for i in range(n_input_ECs):
    gfdl_pred_ECs_range[:,i] = np.subtract(gfdl_pred_ECs_ub[:,i],gfdl_pred_ECs_lb[:,i])
    gfdl_train_2GP = np.c_[gfdl_pred_ECs[:,i],gfdl_pred_ECs_lb[:,i],gfdl_pred_ECs_ub[:,i],gfdl_pred_ECs_range[:,i] ]
    gfdl_train_2GP_sc = scaler_z.fit_transform(gfdl_train_2GP)
    gfdl_pred_ECs_range_ssp126[:,i] = np.subtract(gfdl_pred_ECs_ub_ssp126[:,i],gfdl_pred_ECs_lb_ssp126[:,i])
    gfdl_train_2GP_ssp126 = np.c_[gfdl_pred_ECs_ssp126[:,i],gfdl_pred_ECs_lb_ssp126[:,i],gfdl_pred_ECs_ub_ssp126[:,i],gfdl_pred_ECs_range_ssp126[:,i] ]
    gfdl_train_2GP_sc_ssp126 = scaler_z.fit_transform(gfdl_train_2GP_ssp126 )
    gfdl_pred_ECs_range_ssp370[:,i] = np.subtract(gfdl_pred_ECs_ub_ssp370[:,i],gfdl_pred_ECs_lb_ssp370[:,i])
    gfdl_train_2GP_ssp370 = np.c_[gfdl_pred_ECs_ssp370[:,i],gfdl_pred_ECs_lb_ssp370[:,i],gfdl_pred_ECs_ub_ssp370[:,i],gfdl_pred_ECs_range_ssp370[:,i] ]
    gfdl_train_2GP_sc_ssp370 = scaler_z.fit_transform(gfdl_train_2GP_ssp370 )
    gfdl_pred_ECs_range_ssp585[:,i] = np.subtract(gfdl_pred_ECs_ub_ssp585[:,i],gfdl_pred_ECs_lb_ssp585[:,i])
    gfdl_train_2GP_ssp585 = np.c_[gfdl_pred_ECs_ssp585[:,i],gfdl_pred_ECs_lb_ssp585[:,i],gfdl_pred_ECs_ub_ssp585[:,i],gfdl_pred_ECs_range_ssp585[:,i] ]
    gfdl_train_2GP_sc_ssp585 = scaler_z.fit_transform(gfdl_train_2GP_ssp585 )
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(gfdl_train_2GP_sc[:,0].min(), gfdl_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(gfdl_train_2GP_sc[:, j].min(), gfdl_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(gfdl_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (gfdl_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_gfdl_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_gfdl_eof_2GP[i].training_loss, model_gfdl_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_gfdl_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_gfdl_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_gfdl_eof1_2GP_timeperiod:',opt_gfdl_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_gfdl_eof_2GP[i].predict_y(gfdl_train_2GP_sc, full_cov=False)
    gfdl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(gfdl_pred_mean) / np.mean(obs_train)
    y = (np.std(gfdl_pred_mean) / np.mean(gfdl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    gfdl_pred_ECs_2GP[:,i] =np.squeeze(gfdl_pred_mean)
    print(gfdl_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    gfdl_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    gfdl_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    y_pred_mean_2GP_ssp126, y_pred_var_2GP_ssp126= model_gfdl_eof_2GP[i].predict_y(gfdl_train_2GP_sc_ssp126, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    gfdl_pred_mean_2GP_ssp126 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp126)
    gfdl_pred_ECs_2GP_ssp126[:,i] =np.squeeze(gfdl_pred_mean_2GP_ssp126)
    print(gfdl_pred_ECs_2GP_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    gfdl_pred_ECs_2GP_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    gfdl_pred_ECs_2GP_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_2GP_ssp370, y_pred_var_2GP_ssp370= model_gfdl_eof_2GP[i].predict_y(gfdl_train_2GP_sc_ssp370, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    gfdl_pred_mean_2GP_ssp370 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp370)
    gfdl_pred_ECs_2GP_ssp370[:,i] =np.squeeze(gfdl_pred_mean_2GP_ssp370)
    print(gfdl_pred_ECs_2GP_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    gfdl_pred_ECs_2GP_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    gfdl_pred_ECs_2GP_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_2GP_ssp585, y_pred_var_2GP_ssp585= model_gfdl_eof_2GP[i].predict_y(gfdl_train_2GP_sc_ssp585, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    gfdl_pred_mean_2GP_ssp585 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp585)
    gfdl_pred_ECs_2GP_ssp585[:,i] =np.squeeze(gfdl_pred_mean_2GP_ssp585)
    print(gfdl_pred_ECs_2GP_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    gfdl_pred_ECs_2GP_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    gfdl_pred_ECs_2GP_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,167.77274
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,39.377
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.22768
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.2668, -2.47974, -3.0547..."


The  1  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 22.40600000001723
[-36425.14729426 -35920.90520005 -35945.00971657 ... -35479.63587862
 -35414.45227771 -35364.76782861]
0.9999982458941095 0.8787568756755462
tf.Tensor(
[[0.12659169]
 [0.44806331]
 [0.91077006]
 ...
 [0.07833322]
 [0.08099441]
 [0.08326765]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[ -68410.45132297  -98727.95311174 -164581.8771766  ...  -47963.71087065
  -47968.20209703  -47954.98294326]
tf.Tensor(
[[0.30505153]
 [0.70401854]
 [0.9268653 ]
 ...
 [0.22959557]
 [0.23141436]
 [0.23332188]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[ -76825.15410006 -116831.80678265 -171169.1279505  ...  -53710.32750942
  -53377.23950864  -53048.05758513]
tf.Tensor(
[[0.36931137]
 [0.69106013]
 [0.94284326]
 ...
 [0.18429146]
 [0.19881703]
 [0.21344734]], shape=(2

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,7.04925
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,13.81024
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.52334
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.56792, -4.34868, -4.58236..."


The  2  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 22.40600000001723
[-208.22930577 -340.27492468 -356.94917863 ... -566.0757957  -513.80529736
 -462.7170691 ]
0.999837318609624 0.6903330514403259
tf.Tensor(
[[0.12659169]
 [0.44806331]
 [0.91077006]
 ...
 [0.07833322]
 [0.08099441]
 [0.08326765]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 2443.0398565   6326.65056608 13822.68728894 ...  4089.68824503
  4145.7127655   4193.17230388]
tf.Tensor(
[[0.30505153]
 [0.70401854]
 [0.9268653 ]
 ...
 [0.22959557]
 [0.23141436]
 [0.23332188]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 4684.3315067   9795.36847414 15408.59952235 ...  5305.93613832
  5377.04127872  5445.95126152]
tf.Tensor(
[[0.36931137]
 [0.69106013]
 [0.94284326]
 ...
 [0.18429146]
 [0.19881703]
 [0.21344734]], shape=(28854, 1), dtype=float64) 

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,14.31622
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,12.1267
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.71574
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.82551, -4.70797, -5.04161..."


The  3  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 22.40600000001723
[2384.12182264 2421.236493   2425.49544994 ... 2896.46449391 2885.61438466
 2874.84754818]
1.0000077305993476 0.532738405997154
tf.Tensor(
[[0.12659169]
 [0.44806331]
 [0.91077006]
 ...
 [0.07833322]
 [0.08099441]
 [0.08326765]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -206.80240677 -1620.18093339 -6213.68513378 ...  2272.14752565
  2283.28763645  2293.30243302]
tf.Tensor(
[[0.30505153]
 [0.70401854]
 [0.9268653 ]
 ...
 [0.22959557]
 [0.23141436]
 [0.23332188]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -880.33268138 -2542.66235182 -6433.82774804 ...  1918.59715342
  1993.94806623  2068.06571576]
tf.Tensor(
[[0.36931137]
 [0.69106013]
 [0.94284326]
 ...
 [0.18429146]
 [0.19881703]
 [0.21344734]], shape=(28854, 1), dtype=float64) [[0.01163053

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,5.05734
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,4.94118
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.70433
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.9093, -6.1607, -5.72542..."


The  4  ECs training for obsdata
opt_gfdl_eof1_2GP_timeperiod: 22.40600000001723
[-154.94374043 -297.18037288 -324.18616403 ... -186.3673701  -178.85436774
 -169.43825417]
0.9999542855401272 0.5455328102115106
tf.Tensor(
[[0.12659169]
 [0.44806331]
 [0.91077006]
 ...
 [0.07833322]
 [0.08099441]
 [0.08326765]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[ 2447.15098505  5574.91890003 10015.12525283 ...  1924.71850084
  1954.7080895   1980.23447099]
tf.Tensor(
[[0.30505153]
 [0.70401854]
 [0.9268653 ]
 ...
 [0.22959557]
 [0.23141436]
 [0.23332188]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[ 4062.74897651  7777.93729907 10003.71011042 ...  3295.43717529
  3314.44235585  3334.33469592]
tf.Tensor(
[[0.36931137]
 [0.69106013]
 [0.94284326]
 ...
 [0.18429146]
 [0.19881703]
 [0.21344734]], shape=(28854, 1), dtype=float64)

In [36]:
#合并校正后的两个主成分-
gfdl_ssp126_train_ECs_2GP_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ssp126[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp126[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp126[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp126[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_lb_ssp126[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp126[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp126[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ub_ssp126[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp126[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp126[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp370_train_ECs_2GP_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ssp370[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp370[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp370[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp370[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_lb_ssp370[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp370[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp370[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ub_ssp370[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp370[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp370[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp585_train_ECs_2GP_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ssp585[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp585[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp585[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_ssp585[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_lb_ssp585[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp585[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp585[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
gfdl_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_pred_ECs_2GP_ub_ssp585[:,0].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp585[:,1].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp585[:,2].reshape(28854, 1),gfdl_pred_ECs_2GP_ub_ssp585[:,3].reshape(28854, 1)],axis=1)

# 写入Excel文件
df_mean_merge_2GP_ssp126 = pd.DataFrame(gfdl_ssp126_train_ECs_2GP_SGPRcor,columns = ['gfdl_EC1_2GP','gfdl_EC2_2GP','gfdl_EC3_2GP','gfdl_EC4_2GP'])
df_lb_merge_2GP_ssp126 = pd.DataFrame(gfdl_ssp126_train_ECs_2GP_lb_SGPRcor,columns = ['gfdl_EC1_2GP_lb','gfdl_EC2_2GP_lb','gfdl_EC3_2GP_lb','gfdl_EC4_2GP_lb'])
df_ub_merge_2GP_ssp126 = pd.DataFrame(gfdl_ssp126_train_ECs_2GP_ub_SGPRcor,columns = ['gfdl_EC1_2GP_ub','gfdl_EC2_2GP_ub','gfdl_EC3_2GP_ub','gfdl_EC4_2GP_ub'])
df_mean_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_ssp126_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_ssp126_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_ssp126_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp370 = pd.DataFrame(gfdl_ssp370_train_ECs_2GP_SGPRcor,columns = ['gfdl_EC1_2GP','gfdl_EC2_2GP','gfdl_EC3_2GP','gfdl_EC4_2GP'])
df_lb_merge_2GP_ssp370 = pd.DataFrame(gfdl_ssp370_train_ECs_2GP_lb_SGPRcor,columns = ['gfdl_EC1_2GP_lb','gfdl_EC2_2GP_lb','gfdl_EC3_2GP_lb','gfdl_EC4_2GP_lb'])
df_ub_merge_2GP_ssp370 = pd.DataFrame(gfdl_ssp370_train_ECs_2GP_ub_SGPRcor,columns = ['gfdl_EC1_2GP_ub','gfdl_EC2_2GP_ub','gfdl_EC3_2GP_ub','gfdl_EC4_2GP_ub'])
df_mean_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_ssp370_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_ssp370_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_ssp370_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp585 = pd.DataFrame(gfdl_ssp585_train_ECs_2GP_SGPRcor,columns = ['gfdl_EC1_2GP','gfdl_EC2_2GP','gfdl_EC3_2GP','gfdl_EC4_2GP'])
df_lb_merge_2GP_ssp585 = pd.DataFrame(gfdl_ssp585_train_ECs_2GP_lb_SGPRcor,columns = ['gfdl_EC1_2GP_lb','gfdl_EC2_2GP_lb','gfdl_EC3_2GP_lb','gfdl_EC4_2GP_lb'])
df_ub_merge_2GP_ssp585 = pd.DataFrame(gfdl_ssp585_train_ECs_2GP_ub_SGPRcor,columns = ['gfdl_EC1_2GP_ub','gfdl_EC2_2GP_ub','gfdl_EC3_2GP_ub','gfdl_EC4_2GP_ub'])
df_mean_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_ssp585_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_ssp585_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_ssp585_2GP.xlsx',index=False)

In [38]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_ssp126_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp126_train_SGPRcor_EC1_2GP = data_frame['gfdl_EC1_2GP']
gfdl_ssp126_train_SGPRcor_EC1_2GP_array = np.array(gfdl_ssp126_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC1_2GP_array)
gfdl_ssp126_train_SGPRcor_EC1_2GP_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC1_2GP_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC2_2GP = data_frame['gfdl_EC2_2GP']
gfdl_ssp126_train_SGPRcor_EC2_2GP_array = np.array(gfdl_ssp126_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC2_2GP_array)
gfdl_ssp126_train_SGPRcor_EC2_2GP_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC2_2GP_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC3_2GP = data_frame['gfdl_EC3_2GP']
gfdl_ssp126_train_SGPRcor_EC3_2GP_array = np.array(gfdl_ssp126_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC3_2GP_array)
gfdl_ssp126_train_SGPRcor_EC3_2GP_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC3_2GP_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC4_2GP = data_frame['gfdl_EC4_2GP']
gfdl_ssp126_train_SGPRcor_EC4_2GP_array = np.array(gfdl_ssp126_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC4_2GP_array)
gfdl_ssp126_train_SGPRcor_EC4_2GP_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC4_2GP_tf,tf.float64)
gfdl_ssp126_train_ECs_2GP_SGPRcor=tf.concat([gfdl_ssp126_train_SGPRcor_EC1_2GP_tf,gfdl_ssp126_train_SGPRcor_EC2_2GP_tf,gfdl_ssp126_train_SGPRcor_EC3_2GP_tf,gfdl_ssp126_train_SGPRcor_EC4_2GP_tf],axis=1)
print(gfdl_ssp126_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_ssp370_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp370_train_SGPRcor_EC1_2GP = data_frame['gfdl_EC1_2GP']
gfdl_ssp370_train_SGPRcor_EC1_2GP_array = np.array(gfdl_ssp370_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC1_2GP_array)
gfdl_ssp370_train_SGPRcor_EC1_2GP_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC1_2GP_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC2_2GP = data_frame['gfdl_EC2_2GP']
gfdl_ssp370_train_SGPRcor_EC2_2GP_array = np.array(gfdl_ssp370_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC2_2GP_array)
gfdl_ssp370_train_SGPRcor_EC2_2GP_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC2_2GP_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC3_2GP = data_frame['gfdl_EC3_2GP']
gfdl_ssp370_train_SGPRcor_EC3_2GP_array = np.array(gfdl_ssp370_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC3_2GP_array)
gfdl_ssp370_train_SGPRcor_EC3_2GP_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC3_2GP_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC4_2GP = data_frame['gfdl_EC4_2GP']
gfdl_ssp370_train_SGPRcor_EC4_2GP_array = np.array(gfdl_ssp370_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC4_2GP_array)
gfdl_ssp370_train_SGPRcor_EC4_2GP_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC4_2GP_tf,tf.float64)
gfdl_ssp370_train_ECs_2GP_SGPRcor=tf.concat([gfdl_ssp370_train_SGPRcor_EC1_2GP_tf,gfdl_ssp370_train_SGPRcor_EC2_2GP_tf,gfdl_ssp370_train_SGPRcor_EC3_2GP_tf,gfdl_ssp370_train_SGPRcor_EC4_2GP_tf],axis=1)
print(gfdl_ssp370_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_mean_ssp585_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp585_train_SGPRcor_EC1_2GP = data_frame['gfdl_EC1_2GP']
gfdl_ssp585_train_SGPRcor_EC1_2GP_array = np.array(gfdl_ssp585_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC1_2GP_array)
gfdl_ssp585_train_SGPRcor_EC1_2GP_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC1_2GP_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC2_2GP = data_frame['gfdl_EC2_2GP']
gfdl_ssp585_train_SGPRcor_EC2_2GP_array = np.array(gfdl_ssp585_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC2_2GP_array)
gfdl_ssp585_train_SGPRcor_EC2_2GP_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC2_2GP_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC3_2GP = data_frame['gfdl_EC3_2GP']
gfdl_ssp585_train_SGPRcor_EC3_2GP_array = np.array(gfdl_ssp585_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC3_2GP_array)
gfdl_ssp585_train_SGPRcor_EC3_2GP_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC3_2GP_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC4_2GP = data_frame['gfdl_EC4_2GP']
gfdl_ssp585_train_SGPRcor_EC4_2GP_array = np.array(gfdl_ssp585_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC4_2GP_array)
gfdl_ssp585_train_SGPRcor_EC4_2GP_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC4_2GP_tf,tf.float64)
gfdl_ssp585_train_ECs_2GP_SGPRcor=tf.concat([gfdl_ssp585_train_SGPRcor_EC1_2GP_tf,gfdl_ssp585_train_SGPRcor_EC2_2GP_tf,gfdl_ssp585_train_SGPRcor_EC3_2GP_tf,gfdl_ssp585_train_SGPRcor_EC4_2GP_tf],axis=1)
print(gfdl_ssp585_train_ECs_2GP_SGPRcor)

tf.Tensor(
[[ -68410.45132297    2443.0398565     -206.80240677    2447.15098505]
 [ -98727.95311174    6326.65056608   -1620.18093339    5574.91890003]
 [-164581.8771766    13822.68728894   -6213.68513378   10015.12525283]
 ...
 [ -47963.71087065    4089.68824503    2272.14752565    1924.71850084]
 [ -47968.20209703    4145.7127655     2283.28763645    1954.7080895 ]
 [ -47954.98294326    4193.17230388    2293.30243302    1980.23447099]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[ -76825.15410006    4684.3315067     -880.33268138    4062.74897651]
 [-116831.80678265    9795.36847414   -2542.66235182    7777.93729907]
 [-171169.1279505    15408.59952235   -6433.82774804   10003.71011042]
 ...
 [ -53710.32750942    5305.93613832    1918.59715342    3295.43717529]
 [ -53377.23950864    5377.04127872    1993.94806623    3314.44235585]
 [ -53048.05758513    5445.95126152    2068.06571576    3334.33469592]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[ -79946.11381079    4857.82906733

In [39]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_ssp126_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp126_train_SGPRcor_EC1_2GP_lb = data_frame['gfdl_EC1_2GP_lb']
gfdl_ssp126_train_SGPRcor_EC1_2GP_lb_array = np.array(gfdl_ssp126_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC1_2GP_lb_array)
gfdl_ssp126_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC2_2GP_lb = data_frame['gfdl_EC2_2GP_lb']
gfdl_ssp126_train_SGPRcor_EC2_2GP_lb_array = np.array(gfdl_ssp126_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC2_2GP_lb_array)
gfdl_ssp126_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC3_2GP_lb = data_frame['gfdl_EC3_2GP_lb']
gfdl_ssp126_train_SGPRcor_EC3_2GP_lb_array = np.array(gfdl_ssp126_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC3_2GP_lb_array)
gfdl_ssp126_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC4_2GP_lb = data_frame['gfdl_EC4_2GP_lb']
gfdl_ssp126_train_SGPRcor_EC4_2GP_lb_array = np.array(gfdl_ssp126_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC4_2GP_lb_array)
gfdl_ssp126_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
gfdl_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_ssp126_train_SGPRcor_EC1_2GP_lb_tf,gfdl_ssp126_train_SGPRcor_EC2_2GP_lb_tf,gfdl_ssp126_train_SGPRcor_EC3_2GP_lb_tf,gfdl_ssp126_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(gfdl_ssp126_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_ssp370_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp370_train_SGPRcor_EC1_2GP_lb = data_frame['gfdl_EC1_2GP_lb']
gfdl_ssp370_train_SGPRcor_EC1_2GP_lb_array = np.array(gfdl_ssp370_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC1_2GP_lb_array)
gfdl_ssp370_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC2_2GP_lb = data_frame['gfdl_EC2_2GP_lb']
gfdl_ssp370_train_SGPRcor_EC2_2GP_lb_array = np.array(gfdl_ssp370_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC2_2GP_lb_array)
gfdl_ssp370_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC3_2GP_lb = data_frame['gfdl_EC3_2GP_lb']
gfdl_ssp370_train_SGPRcor_EC3_2GP_lb_array = np.array(gfdl_ssp370_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC3_2GP_lb_array)
gfdl_ssp370_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC4_2GP_lb = data_frame['gfdl_EC4_2GP_lb']
gfdl_ssp370_train_SGPRcor_EC4_2GP_lb_array = np.array(gfdl_ssp370_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC4_2GP_lb_array)
gfdl_ssp370_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
gfdl_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_ssp370_train_SGPRcor_EC1_2GP_lb_tf,gfdl_ssp370_train_SGPRcor_EC2_2GP_lb_tf,gfdl_ssp370_train_SGPRcor_EC3_2GP_lb_tf,gfdl_ssp370_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(gfdl_ssp370_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_lb_ssp585_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp585_train_SGPRcor_EC1_2GP_lb = data_frame['gfdl_EC1_2GP_lb']
gfdl_ssp585_train_SGPRcor_EC1_2GP_lb_array = np.array(gfdl_ssp585_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC1_2GP_lb_array)
gfdl_ssp585_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC2_2GP_lb = data_frame['gfdl_EC2_2GP_lb']
gfdl_ssp585_train_SGPRcor_EC2_2GP_lb_array = np.array(gfdl_ssp585_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC2_2GP_lb_array)
gfdl_ssp585_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC3_2GP_lb = data_frame['gfdl_EC3_2GP_lb']
gfdl_ssp585_train_SGPRcor_EC3_2GP_lb_array = np.array(gfdl_ssp585_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC3_2GP_lb_array)
gfdl_ssp585_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC4_2GP_lb = data_frame['gfdl_EC4_2GP_lb']
gfdl_ssp585_train_SGPRcor_EC4_2GP_lb_array = np.array(gfdl_ssp585_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC4_2GP_lb_array)
gfdl_ssp585_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
gfdl_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([gfdl_ssp585_train_SGPRcor_EC1_2GP_lb_tf,gfdl_ssp585_train_SGPRcor_EC2_2GP_lb_tf,gfdl_ssp585_train_SGPRcor_EC3_2GP_lb_tf,gfdl_ssp585_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(gfdl_ssp585_train_ECs_2GP_lb_SGPRcor)

tf.Tensor(
[[-377034.92954065  -27491.04451381  -18574.74630251  -11599.47806907]
 [-332872.28456211  -22051.87847034  -14607.65013857   -8895.60116694]
 [-272342.5243794   -14596.90342605   -9170.30870347   -5189.64060844]
 ...
 [-383840.03804698  -28329.1762681   -19186.04392437  -12016.12375946]
 [-383472.86455968  -28283.95439415  -19153.06101464  -11993.64340504]
 [-383158.74186985  -28245.26637017  -19124.84361691  -11974.41110847]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-352296.06278506  -24444.15264227  -16352.4730792   -10084.83032916]
 [-297815.06417972  -17734.15604283  -11458.48720765   -6749.20779555]
 [-271847.73560069  -14535.9641792    -9125.86221104   -5159.346953  ]
 ...
 [-362619.93434883  -25715.66279919  -17279.8584588   -10716.91379869]
 [-362370.38960665  -25684.9283337   -17257.44204805  -10701.63531469]
 [-362108.51653775  -25652.67548506  -17233.91819328  -10685.6020236 ]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-343427.12520755  -23351.83527458

In [40]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_ssp126_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp126_train_SGPRcor_EC1_2GP_ub = data_frame['gfdl_EC1_2GP_ub']
gfdl_ssp126_train_SGPRcor_EC1_2GP_ub_array = np.array(gfdl_ssp126_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC1_2GP_ub_array)
gfdl_ssp126_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC2_2GP_ub = data_frame['gfdl_EC2_2GP_ub']
gfdl_ssp126_train_SGPRcor_EC2_2GP_ub_array = np.array(gfdl_ssp126_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC2_2GP_ub_array)
gfdl_ssp126_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC3_2GP_ub = data_frame['gfdl_EC3_2GP_ub']
gfdl_ssp126_train_SGPRcor_EC3_2GP_ub_array = np.array(gfdl_ssp126_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC3_2GP_ub_array)
gfdl_ssp126_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
gfdl_ssp126_train_SGPRcor_EC4_2GP_ub = data_frame['gfdl_EC4_2GP_ub']
gfdl_ssp126_train_SGPRcor_EC4_2GP_ub_array = np.array(gfdl_ssp126_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp126_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp126_train_SGPRcor_EC4_2GP_ub_array)
gfdl_ssp126_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(gfdl_ssp126_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
gfdl_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_ssp126_train_SGPRcor_EC1_2GP_ub_tf,gfdl_ssp126_train_SGPRcor_EC2_2GP_ub_tf,gfdl_ssp126_train_SGPRcor_EC3_2GP_ub_tf,gfdl_ssp126_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(gfdl_ssp126_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_ssp370_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp370_train_SGPRcor_EC1_2GP_ub = data_frame['gfdl_EC1_2GP_ub']
gfdl_ssp370_train_SGPRcor_EC1_2GP_ub_array = np.array(gfdl_ssp370_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC1_2GP_ub_array)
gfdl_ssp370_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC2_2GP_ub = data_frame['gfdl_EC2_2GP_ub']
gfdl_ssp370_train_SGPRcor_EC2_2GP_ub_array = np.array(gfdl_ssp370_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC2_2GP_ub_array)
gfdl_ssp370_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC3_2GP_ub = data_frame['gfdl_EC3_2GP_ub']
gfdl_ssp370_train_SGPRcor_EC3_2GP_ub_array = np.array(gfdl_ssp370_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC3_2GP_ub_array)
gfdl_ssp370_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
gfdl_ssp370_train_SGPRcor_EC4_2GP_ub = data_frame['gfdl_EC4_2GP_ub']
gfdl_ssp370_train_SGPRcor_EC4_2GP_ub_array = np.array(gfdl_ssp370_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp370_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp370_train_SGPRcor_EC4_2GP_ub_array)
gfdl_ssp370_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(gfdl_ssp370_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
gfdl_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_ssp370_train_SGPRcor_EC1_2GP_ub_tf,gfdl_ssp370_train_SGPRcor_EC2_2GP_ub_tf,gfdl_ssp370_train_SGPRcor_EC3_2GP_ub_tf,gfdl_ssp370_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(gfdl_ssp370_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\gfdl_ECs_pred_ub_ssp585_2GP.xlsx')
# 获取指定列的数据
gfdl_ssp585_train_SGPRcor_EC1_2GP_ub = data_frame['gfdl_EC1_2GP_ub']
gfdl_ssp585_train_SGPRcor_EC1_2GP_ub_array = np.array(gfdl_ssp585_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC1_2GP_ub_array)
gfdl_ssp585_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC2_2GP_ub = data_frame['gfdl_EC2_2GP_ub']
gfdl_ssp585_train_SGPRcor_EC2_2GP_ub_array = np.array(gfdl_ssp585_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC2_2GP_ub_array)
gfdl_ssp585_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC3_2GP_ub = data_frame['gfdl_EC3_2GP_ub']
gfdl_ssp585_train_SGPRcor_EC3_2GP_ub_array = np.array(gfdl_ssp585_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC3_2GP_ub_array)
gfdl_ssp585_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
gfdl_ssp585_train_SGPRcor_EC4_2GP_ub = data_frame['gfdl_EC4_2GP_ub']
gfdl_ssp585_train_SGPRcor_EC4_2GP_ub_array = np.array(gfdl_ssp585_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
gfdl_ssp585_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(gfdl_ssp585_train_SGPRcor_EC4_2GP_ub_array)
gfdl_ssp585_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(gfdl_ssp585_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
gfdl_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([gfdl_ssp585_train_SGPRcor_EC1_2GP_ub_tf,gfdl_ssp585_train_SGPRcor_EC2_2GP_ub_tf,gfdl_ssp585_train_SGPRcor_EC3_2GP_ub_tf,gfdl_ssp585_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(gfdl_ssp585_train_ECs_2GP_ub_SGPRcor)

tf.Tensor(
[[ 98566.78991571  31085.08297898  24148.18747412  17519.4414267 ]
 [143902.86901737  36668.77168498  28220.6923157   20295.16233688]
 [212192.14017436  45079.42473782  34355.06460651  24476.20228669]
 ...
 [ 91936.58675328  30268.49294186  23552.60146371  17113.50439109]
 [ 92310.29557104  30314.51972111  23586.17143712  17136.38487393]
 [ 92629.05118957  30353.77834652  23614.80500723  17155.90082354]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[123511.71082752  34157.35292219  26388.97038255  19046.7049208 ]
 [180104.39895408  41127.42973069  31472.64768682  22511.6165521 ]
 [216178.31893116  45570.37106016  34713.13995419  24720.25779273]
 ...
 [112828.40376985  32841.57391919  25429.29719116  18392.61485882]
 [113085.21475589  32873.20331085  25452.36632296  18408.33822171]
 [113354.40041181  32906.35679339  25476.54706176  18424.81922908]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[132532.91775822  35268.4241099   27199.33840798  19599.03218882]
 [178143.0619193

In [41]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((28854,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [42]:
recondata_SGPR_gfdl_mix4_2GP_ssp126 = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp126_train_ECs_2GP_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp370 = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp370_train_ECs_2GP_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp585 = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp585_train_ECs_2GP_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp126_lb = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp126_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp370_lb = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp370_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp585_lb = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp585_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp126_ub = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp126_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp370_ub = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp370_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_gfdl_mix4_2GP_ssp585_ub = ReconstructDataFromEOFandECs(obsdata_eofs,gfdl_ssp585_train_ECs_2GP_ub_SGPRcor)

In [43]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2100, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 2022-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [44]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc1 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp126.nc' 
out_file_nc2 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp370.nc' 
out_file_nc3 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp585.nc'
out_file_nc4 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp126_lb.nc' 
out_file_nc5 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp370_lb.nc' 
out_file_nc6 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp585_lb.nc' 
out_file_nc7 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp126_ub.nc' 
out_file_nc8 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp370_ub.nc' 
out_file_nc9 = out_dir+'recondata_SGPR_gfdl_mix4_2GP_ssp585_ub.nc' 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp126,out_file_nc1) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp370,out_file_nc2) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp585,out_file_nc3) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp126_lb,out_file_nc4) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp370_lb,out_file_nc5) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp585_lb,out_file_nc6) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp126_ub,out_file_nc7) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp370_ub,out_file_nc8) 
writeNetCDF(recondata_SGPR_gfdl_mix4_2GP_ssp585_ub,out_file_nc9) 